<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/api_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xmltodict

In [2]:
import pandas as pd
import re
import sys
import requests
import json
import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen
import xmltodict
from tqdm import tqdm
import random
import pprint

In [3]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)
totalCnt = int(dict_contents['LawSearch']['totalCnt'])

In [4]:
totalCnt

2623

In [ ]:
rows = []
not_match_jo = []
changed_num = 0
changed = []
eng_cnt = 0
kor_cnt = 0
total_page = int(totalCnt/20)+1
for p in tqdm(range(1,total_page)):
  url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=' + str(p)
  # url 불러오기
  response = requests.get(url)
  # XML -> 딕셔너리 변환후 파싱
  contents = response.text
  dict_contents = xmltodict.parse(contents)
  try:
    for content in dict_contents['LawSearch']['law']:

      name_kor = content['법령명한글']
      name_eng = content['법령명영문']
      mst = content['법령일련번호']

      eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      eng_response = requests.get(eng_url)
      eng_contents = eng_response.text
      eng_dict_contents = xmltodict.parse(eng_contents)

      kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      kor_response = requests.get(kor_url)
      kor_contents = kor_response.text
      kor_dict_contents = xmltodict.parse(kor_contents)

      # # 영어조문 개수와 한글 조문 개수 맞지 않다면 건너뛰기
      # if len(eng_dict_contents['Law']['JoSection']['Jo']) != len(kor_dict_contents['법령']['조문']['조문단위']):
      #     not_match_jo += 1
      #     print(f'영한 조문 개수 맞지 않는 법령 개수', not_match_jo, kor_dict_contents['법령']['기본정보']['법령명_한글'])
      #     continue
      # eng_cnt += len(eng_dict_contents['Law']['JoSection']['Jo'])
      # kor_cnt += len(kor_dict_contents['법령']['조문']['조문단위'])
      min_num_jo = min(len(eng_dict_contents['Law']['JoSection']['Jo']), len(kor_dict_contents['법령']['조문']['조문단위']))
      temp_cnt = 0
      print('법령명:', name_kor, ', 조문 개수:', min_num_jo, ', 페이지:', p, ', 몇번째?:', content['@id'])
      try:
        # for eng_jo, kor_jo in zip(eng_dict_contents['Law']['JoSection']['Jo'],kor_dict_contents['법령']['조문']['조문단위']):
        for ind in range(min_num_jo):
          eng_jo, kor_jo = eng_dict_contents['Law']['JoSection']['Jo'][ind], kor_dict_contents['법령']['조문']['조문단위'][ind]

          if kor_jo['조문여부']!='조문':
            continue
          # if kor_jo.get('조문가지번호'):
          #   continue

          kor_total = 0
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict:
              kor_total += len(kor_jo['항']['호'])
              for ho in kor_jo['항']['호']:
                if ho.get('목'):
                  kor_total += len(ho['목'])
              kor_total += 1 # 호부터 시작하면 영어에서는 또 \n이 하나 늘어남
            else:
              kor_total += len(kor_jo['항'])
              for hang in kor_jo['항']:
                if hang.get('호'):
                  kor_total += len(hang['호'])
                  for ho in hang['호']:
                    if ho.get('목'):
                      kor_total += len(ho['목'])
          else:
            kor_total = 1

          if kor_jo.get('조문참고자료'):
            if '\n\t\t\n\t\t' in kor_jo['조문참고자료']:
              kor_total += 2
              kor_jo['조문참고자료'] = kor_jo['조문참고자료'].replace('\n\t\t\n\t\t', '\n')
            else:
              kor_total += 1

          if (eng_jo['joCts'].count('\n')+1) != kor_total:
            not_match_jo.append([name_kor, kor_jo['조문제목']])
            continue

          if eng_jo['joYn']!= 'Y':
            changed_num += 1
            changed.append([eng_jo, kor_jo])
            print('안바뀐 법령 개수:', changed_num)
            continue
          # if eng_jo['joBrNo']!='00':
          #   continue

          eng_text = eng_jo['joCts']

          jo_txt = ''
          hang_txt = ''
          ho_txt = ''
          mok_txt = ''

          jo_txt += kor_jo['조문내용']
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict: # 바로 호내용일 때
              for ho in kor_jo['항']['호']:
                ho_txt += '\n' + ho['호내용']
                if ho.get('목'):
                  for mok in ho['목']:
                    mok_txt += '\n' + mok['목내용']
                  ho_txt += mok_txt
              hang_txt += ho_txt
            else:
              for hang in kor_jo['항']:
                if hang_txt == '':
                  hang_txt += hang['항내용'] # 영어의 경우 맨 첫항과 조문 내용 사이에 \n으로 분리가 안돼있음
                else:
                  hang_txt += '\n' + hang['항내용']

                if hang.get('호'):
                  for ho in hang['호']:
                    ho_txt += '\n' + ho['호내용']
                    if ho.get('목'):
                      for mok in ho['목']:
                        mok_txt += '\n' + mok['목내용']
                      ho_txt += mok_txt
                  hang_txt += ho_txt
              jo_txt += hang_txt
          if kor_jo.get('조문참고자료'):
            eng_text = '\n'.join([i.strip() for i in eng_text.split('\n')[:-1]]) # 조문참고자료 부분 삭제하기
          kor_text = jo_txt
          # rows.append({'name_kor':name_kor, 'name_eng':name_eng, 'MST':mst, 'eng':eng_text, 'kor':kor_text})
        changed.append(content['현행연혁코드'])
      except Exception as e:
        print('eng_kor_contents 예외 발생:', e)
  except Exception as e:
    print('dict_contents 예외 발생:', e)
# print(len(not_match_jo))
# df = pd.DataFrame(rows)

  0%|          | 0/131 [00:00<?, ?it/s]

법령명: 119구조ㆍ구급에 관한 법률 , 조문 개수: 48 , 페이지: 1 , 몇번째?: 1
법령명: 119구조ㆍ구급에 관한 법률 시행령 , 조문 개수: 47 , 페이지: 1 , 몇번째?: 2
법령명: 1인 창조기업 육성에 관한 법률 , 조문 개수: 24 , 페이지: 1 , 몇번째?: 3
법령명: 1인 창조기업 육성에 관한 법률 시행령 , 조문 개수: 12 , 페이지: 1 , 몇번째?: 4
법령명: 2011대구세계육상선수권대회, 2013충주세계조정선수권대회, 2014인천아시아경기대회, 2014인천장애인아시아경기대회 및 2015광주하계유니버시아드대회 지원법 , 조문 개수: 46 , 페이지: 1 , 몇번째?: 5
법령명: 2018 평창 동계올림픽대회 및 동계패럴림픽대회 지원 등에 관한 특별법 , 조문 개수: 113 , 페이지: 1 , 몇번째?: 6
법령명: 2018 평창 동계올림픽대회 및 동계패럴림픽대회 지원 등에 관한 특별법 시행령 , 조문 개수: 64 , 페이지: 1 , 몇번째?: 7
법령명: 2023 새만금 세계스카우트잼버리 지원 특별법 , 조문 개수: 39 , 페이지: 1 , 몇번째?: 8
법령명: 2023 새만금 세계스카우트잼버리 지원 특별법 시행령 , 조문 개수: 12 , 페이지: 1 , 몇번째?: 9
법령명: 4ㆍ16세월호참사 진상규명 및 안전사회 건설 등을 위한 특별법 , 조문 개수: 60 , 페이지: 1 , 몇번째?: 10
법령명: 4ㆍ16세월호참사 피해구제 및 지원 등을 위한 특별법 , 조문 개수: 55 , 페이지: 1 , 몇번째?: 11
법령명: 4ㆍ16세월호참사 피해구제 및 지원 등을 위한 특별법 시행령 , 조문 개수: 42 , 페이지: 1 , 몇번째?: 12
법령명: 5ㆍ18민주유공자예우 및 단체설립에 관한 법률 , 조문 개수: 125 , 페이지: 1 , 몇번째?: 13
법령명: 5ㆍ18민주화운동 등에 관한 특별법 , 조문 개수: 9 , 페이지: 1 , 몇번째?: 14
법령명: 5ㆍ18민주화운동 진상규명을 위한 특별법 , 조문 개수: 84 

  1%|          | 1/131 [01:28<3:12:10, 88.69s/it]

법령명: 6ㆍ25전쟁 전후 적 지역에서 활동한 비정규군 공로자 보상에 관한 법률 , 조문 개수: 21 , 페이지: 1 , 몇번째?: 20
법령명: 6ㆍ25전쟁 전후 적 지역에서 활동한 비정규군 공로자 보상에 관한 법률 시행령 , 조문 개수: 20 , 페이지: 2 , 몇번째?: 21
법령명: 가등기담보 등에 관한 법률 , 조문 개수: 18 , 페이지: 2 , 몇번째?: 22
법령명: 가맹사업거래의 공정화에 관한 법률 , 조문 개수: 75 , 페이지: 2 , 몇번째?: 23
법령명: 가맹사업거래의 공정화에 관한 법률 시행령 , 조문 개수: 62 , 페이지: 2 , 몇번째?: 24
법령명: 가맹사업 진흥에 관한 법률 , 조문 개수: 19 , 페이지: 2 , 몇번째?: 25
법령명: 가맹사업 진흥에 관한 법률 시행령 , 조문 개수: 7 , 페이지: 2 , 몇번째?: 26
법령명: 가사근로자의 고용개선 등에 관한 법률 , 조문 개수: 35 , 페이지: 2 , 몇번째?: 27
법령명: 가사소송법 , 조문 개수: 105 , 페이지: 2 , 몇번째?: 28
법령명: 가상자산 이용자 보호 등에 관한 법률 , 조문 개수: 27 , 페이지: 2 , 몇번째?: 29
법령명: 가석방자관리규정 , 조문 개수: 21 , 페이지: 2 , 몇번째?: 30
법령명: 가습기살균제 피해구제를 위한 특별법 , 조문 개수: 58 , 페이지: 2 , 몇번째?: 31
법령명: 가습기살균제 피해구제를 위한 특별법 시행령 , 조문 개수: 54 , 페이지: 2 , 몇번째?: 32
법령명: 가정의례에관한법률 , 조문 개수: 15 , 페이지: 2 , 몇번째?: 33
법령명: 가정폭력방지 및 피해자보호 등에 관한 법률 , 조문 개수: 42 , 페이지: 2 , 몇번째?: 34
법령명: 가정폭력방지 및 피해자보호 등에 관한 법률 시행령 , 조문 개수: 14 , 페이지: 2 , 몇번째?: 35
법령명: 가정폭력범죄의 처벌 등에 관한 특례법 , 조문 개수: 89 , 페이지: 2 , 몇번째?: 36
법령명: 가정폭력범죄의

  2%|▏         | 2/131 [02:59<3:13:08, 89.83s/it]

...

법령명: 항공ㆍ철도 사고조사에 관한 법률 시행령 , 조문 개수: 9 , 페이지: 123 , 몇번째?: 2460
법령명: 항로표지법 , 조문 개수: 72 , 페이지: 124 , 몇번째?: 2461
법령명: 항로표지법 시행령 , 조문 개수: 37 , 페이지: 124 , 몇번째?: 2462
법령명: 항만법 , 조문 개수: 121 , 페이지: 124 , 몇번째?: 2463
법령명: 항만안전특별법 , 조문 개수: 17 , 페이지: 124 , 몇번째?: 2464
법령명: 항만안전특별법 시행령 , 조문 개수: 11 , 페이지: 124 , 몇번째?: 2465
법령명: 항만운송사업법 , 조문 개수: 64 , 페이지: 124 , 몇번째?: 2466
법령명: 항만운송사업법 시행령 , 조문 개수: 37 , 페이지: 124 , 몇번째?: 2467
법령명: 항만인력공급체제의 개편을 위한 지원특별법 , 조문 개수: 14 , 페이지: 124 , 몇번째?: 2468
법령명: 항만 재개발 및 주변지역 발전에 관한 법률 , 조문 개수: 58 , 페이지: 124 , 몇번째?: 2469
법령명: 항만지역등 대기질 개선에 관한 특별법 , 조문 개수: 30 , 페이지: 124 , 몇번째?: 2470
법령명: 항만지역등 대기질 개선에 관한 특별법 시행령 , 조문 개수: 13 , 페이지: 124 , 몇번째?: 2471


 95%|█████████▍| 124/131 [4:51:51<10:39, 91.38s/it] 

dict_contents 예외 발생: 'NoneType' object is not subscriptable
법령명: 해양공간계획 및 관리에 관한 법률 , 조문 개수: 27 , 페이지: 125 , 몇번째?: 2481
법령명: 해양과학조사법 , 조문 개수: 31 , 페이지: 125 , 몇번째?: 2482
법령명: 해양과학조사법 시행령 , 조문 개수: 23 , 페이지: 125 , 몇번째?: 2483
법령명: 해양교육 및 해양문화의 활성화에 관한 법률 , 조문 개수: 34 , 페이지: 125 , 몇번째?: 2484
법령명: 해양교육 및 해양문화의 활성화에 관한 법률 시행령 , 조문 개수: 23 , 페이지: 125 , 몇번째?: 2485
법령명: 해양사고의 조사 및 심판에 관한 법률 , 조문 개수: 141 , 페이지: 125 , 몇번째?: 2486
법령명: 해양사고의 조사 및 심판에 관한 법률 시행령 , 조문 개수: 109 , 페이지: 125 , 몇번째?: 2487
법령명: 해양산업클러스터의 지정 및 육성 등에 관한 특별법 , 조문 개수: 50 , 페이지: 125 , 몇번째?: 2488
법령명: 해양생태계의 보전 및 관리에 관한 법률 , 조문 개수: 81 , 페이지: 125 , 몇번째?: 2489
법령명: 해양수산과학기술 육성법 , 조문 개수: 30 , 페이지: 125 , 몇번째?: 2490
법령명: 해양수산과학기술 육성법 시행령 , 조문 개수: 23 , 페이지: 125 , 몇번째?: 2491
법령명: 해양수산발전 기본법 , 조문 개수: 47 , 페이지: 125 , 몇번째?: 2492
법령명: 해양수산발전 기본법 시행령 , 조문 개수: 34 , 페이지: 125 , 몇번째?: 2493
eng_kor_contents 예외 발생: '조문제목'
법령명: 해양수산부 소관 친환경농어업 육성 및 유기식품 등의 관리ㆍ지원에 관한 법률 시행규칙 , 조문 개수: 55 , 페이지: 125 , 몇번째?: 2494
법령명: 해양수산생명자원의 확보ㆍ관리 및 이용 등에 관한 법률 , 조문 개수:

 95%|█████████▌| 125/131 [4:53:51<10:00, 100.07s/it]

법령명: 해양치유자원의 관리 및 활용에 관한 법률 , 조문 개수: 44 , 페이지: 125 , 몇번째?: 2500
법령명: 해양폐기물 및 해양오염퇴적물 관리법 , 조문 개수: 46 , 페이지: 126 , 몇번째?: 2501


In [6]:
# {'시행예정', '연혁', '현행'}
print(changed.count('시행예정'))
print(changed.count('연혁'))
print(changed.count('현행'))

5219
48319
30751


In [ ]:
df.to_csv('/content/drive/MyDrive/KB/KB_trans/120-.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=1'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)

content = dict_contents['LawSearch']['law'][1]

name_kor = content['법령명한글']
name_eng = content['법령명영문']
mst = content['법령일련번호']
print(name_kor)
eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
eng_response = requests.get(eng_url)
eng_contents = eng_response.text
eng_dict_contents = xmltodict.parse(eng_contents)

kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
kor_response = requests.get(kor_url)
kor_contents = kor_response.text
kor_dict_contents = xmltodict.parse(kor_contents)

119구조ㆍ구급에 관한 법률 시행령


In [ ]:
count = 0
for temp in kor_dict_contents['법령']['조문']['조문단위']:
    if temp.get('조문제목'):
        if temp['조문제목'] == '기념화폐의 판매':
            print(count)
    count += 1

17


In [ ]:
eng_dict_contents['Law']['JoSection']['Jo'][7]

{'@No': '8',
 'chapNo': '00000300000000000000',
 'joNo': '0005',
 'joBrNo': '00',
 'joYn': 'Y',
 'joTtl': 'Organization and Operation of 119 Rescue Squads',
 'joCts': 'Article 5 (Organization and Operation of 119 Rescue Squads) (1) Any 119 rescue squads under Article 8 (1) of the Act (hereinafter referred to as “rescue squad”) shall be organized and operated by classifications as follows: <Amended on Jul. 7, 2014; Jul. 14, 2014; Nov. 19, 2014; Oct. 25, 2016; Jul. 26, 2017>\n  1. General rescue squad: At least one squad shall be established in each fire station as prescribed by Municipal Rule of a City/Do, but where a Si/Gun/Gu (a Gu refers to an autonomous Gu; hereinafter the same apply) does not have a fire station, the squad may be established in an 119 safety center located in the center of the relevant Si/Gun/Gu;\n  2. Special rescue squad: A squad shall be established in a fire station having jurisdiction over an area classified in the following items as prescribed by Municipal Ru

In [ ]:
kor_dict_contents['법령']['조문']['조문단위'][7]

{'@조문키': '0005001',
 '조문번호': '5',
 '조문여부': '조문',
 '조문제목': '119구조대의 편성과 운영',
 '조문시행일자': '20210121',
 '조문이동이전': '000000',
 '조문이동이후': '000000',
 '조문변경여부': 'N',
 '조문내용': '제5조(119구조대의 편성과 운영)',
 '항': [{'항번호': '①',
   '항내용': '① 법 제8조제1항에 따른 119구조대(이하 "구조대"라 한다)는 다음 각 호의 구분에 따라 편성·운영한다. <개정 2014.7.7, 2014.7.14, 2014.11.19, 2016.10.25, 2017.7.26>',
   '호': [{'호번호': '1.',
     '호내용': '1. 일반구조대: 시·도의 규칙으로 정하는 바에 따라 소방서마다 1개 대(隊) 이상 설치하되, 소방서가 없는 시·군·구(자치구를 말한다. 이하 같다)의 경우에는 해당 시·군·구 지역의 중심지에 있는 119안전센터에 설치할 수 있다.'},
    {'호번호': '2.',
     '호내용': '2. 특수구조대: 소방대상물, 지역 특성, 재난 발생 유형 및 빈도 등을 고려하여 시·도의 규칙으로 정하는 바에 따라 다음 각 목의 구분에 따른 지역을 관할하는 소방서에 다음 각 목의 구분에 따라 설치한다. 다만, 라목에 따른 고속국도구조대는 제3호에 따라 설치되는 직할구조대에 설치할 수 있다.',
     '목': [{'목번호': '가.', '목내용': '가. 화학구조대: 화학공장이 밀집한 지역'},
      {'목번호': '나.', '목내용': '나. 수난구조대: 「내수면어업법」 제2조제1호에 따른 내수면지역'},
      {'목번호': '다.', '목내용': '다. 산악구조대: 「자연공원법」 제2조제1호에 따른 자연공원 등 산악지역'},
      {'목번호': '라.',
       '목내용': '라. 고속국도구조대: 「도로법」 제10조제1호에 따른 고속국도(이하 "고속국도"라 한다)'},
   

In [ ]:
# 영한 조문 개수 맞지 않는 법령 개수 2 간접투자자산 운용업법 시행령 17
# 영한 조문 개수 맞지 않는 법령 개수 3 간척지의 농어업적 이용 및 관리에 관한 법률 9
# 영한 조문 개수 맞지 않는 법령 개수 4 개발이익 환수에 관한 법률 14
# 영한 조문 개수 맞지 않는 법령 개수 5 개별소비세법 18

In [ ]:
kor_jo = kor_dict_contents['법령']['조문']['조문단위'][43]
eng_jo = eng_dict_contents['Law']['JoSection']['Jo'][43]
kor_total = 0
if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
  if type(kor_jo['항']) == dict:
    kor_total += len(kor_jo['항']['호'])
    for ho in kor_jo['항']['호']:
      if ho.get('목'):
        kor_total += len(ho['목'])
    kor_total += 1
  else:
    kor_total += len(kor_jo['항'])
    for hang in kor_jo['항']:
      if hang.get('호'):
        kor_total += len(hang['호'])
        for ho in hang['호']:
          if ho.get('목'):
            kor_total += len(ho['목'])
else:
  kor_total = 1
print(kor_total)

if kor_jo.get('조문참고자료'):
  kor_total += (1 + kor_jo['조문참고자료'].count('\n'))
print(kor_total)
if (eng_jo['joCts'].count('\n')+1) != kor_total:
  not_match_jo += 1
  print(not_match_jo)

6
9
4
